In [6]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

from torch.autograd import Variable
from torch.autograd import Function
import torch.backends.cudnn as cudnn
import os
import numpy as np
from tqdm import tqdm
from model import FaceModel,FaceModelCenter,FaceModelSoftmax
from eval_metrics import evaluate
#from logger import Logger
from LFWDataset import LFWDataset
from PIL import Image
from utils import PairwiseDistance,display_triplet_distance,display_triplet_distance_test
import collections

In [7]:
# Training settings
parser = argparse.ArgumentParser(description='PyTorch Face Recognition')
# Model options
parser.add_argument('--dataroot', type=str, default='/home/jw4937/datasets/casia/train',#default='/media/lior/LinuxHDD/datasets/vgg_face_dataset/aligned'
                    help='path to dataset')
parser.add_argument('--testdataroot', type=str, default='/home/jw4937/datasets/casia/test',#default='/media/lior/LinuxHDD/datasets/vgg_face_dataset/aligned'
                    help='path to dataset')
parser.add_argument('--lfw-dir', type=str, default='/scratch/jw4937/deeplearningdataset/lfw',
                    help='path to dataset')
parser.add_argument('--lfw-pairs-path', type=str, default='lfw_pairs.txt',
                    help='path to pairs file')

parser.add_argument('--log-dir', default='/scratch/jw4937/logdir_center_loss',
                    help='folder to output model checkpoints')

parser.add_argument('--resume',
                    default='/home/jw4937/logdir/run-optim_adam-lr0.001-wd0.0-embeddings512-center0.5-MSCeleb/checkpoint_11.pth',
                    type=str, metavar='PATH',
                    help='path to latest checkpoint (default: none)')
parser.add_argument('--start-epoch', default=0, type=int, metavar='N',
                    help='manual epoch number (useful on restarts)')
parser.add_argument('--epochs', type=int, default=100, metavar='E',
                    help='number of epochs to train (default: 10)')
# Training options
# parser.add_argument('--embedding-size', type=int, default=256, metavar='ES',
#                     help='Dimensionality of the embedding')

parser.add_argument('--center_loss_weight', type=float, default=0.5, help='weight for center loss')
parser.add_argument('--alpha', type=float, default=0.5, help='learning rate of the centers')
parser.add_argument('--embedding-size', type=int, default=512, metavar='ES',
                    help='Dimensionality of the embedding')

parser.add_argument('--batch-size', type=int, default=64, metavar='BS',
                    help='input batch size for training (default: 128)')
parser.add_argument('--test-batch-size', type=int, default=64, metavar='BST',
                    help='input batch size for testing (default: 1000)')

parser.add_argument('--lr', type=float, default=0.001, metavar='LR',
                    help='learning rate (default: 0.001)')
parser.add_argument('--beta1', type=float, default=0.5, help='beta1 for adam. default=0.5')

parser.add_argument('--lr-decay', default=1e-4, type=float, metavar='LRD',
                    help='learning rate decay ratio (default: 1e-4')
parser.add_argument('--wd', default=0.0, type=float,
                    metavar='W', help='weight decay (default: 0.0)')
parser.add_argument('--optimizer', default='adam', type=str,
                    metavar='OPT', help='The optimizer to use (default: Adagrad)')
# Device options
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='enables CUDA training')
parser.add_argument('--gpu-id', default='0', type=str,
                    help='id(s) for CUDA_VISIBLE_DEVICES')
parser.add_argument('--seed', type=int, default=0, metavar='S',
                    help='random seed (default: 0)')
parser.add_argument('--log-interval', type=int, default=10, metavar='LI',
                    help='how many batches to wait before logging training status')

args = parser.parse_args(args = [])

In [8]:
# set the device to use by setting CUDA_VISIBLE_DEVICES env variable in
# order to prevent any memory allocation on unused GPUs
os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu_id

args.cuda = not args.no_cuda and torch.cuda.is_available()
np.random.seed(args.seed)

if not os.path.exists(args.log_dir):
    os.makedirs(args.log_dir)

if args.cuda:
    cudnn.benchmark = True

LOG_DIR = args.log_dir + '/run-optim_{}-lr{}-wd{}-embeddings{}-center_loss{}-MSCeleb'.format(args.optimizer, args.lr, args.wd,args.embedding_size,args.center_loss_weight)


# create logger
#logger = Logger(LOG_DIR)

kwargs = {'num_workers': 2, 'pin_memory': True} if args.cuda else {}
l2_dist = PairwiseDistance(2)

transform = transforms.Compose([
                         transforms.Resize(96),
                         transforms.RandomHorizontalFlip(),
                         transforms.ToTensor(),
                         transforms.Normalize(mean = [ 0.5, 0.5, 0.5 ],
                                               std = [ 0.5, 0.5, 0.5 ])
                     ])

train_dir = ImageFolder(args.dataroot,transform=transform)
testacc_dir = ImageFolder(args.testdataroot,transform=transform)
train_loader = torch.utils.data.DataLoader(train_dir,
    batch_size=args.batch_size, shuffle=True, **kwargs)

testaccuracy_loader = torch.utils.data.DataLoader(testacc_dir,
    batch_size=args.batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    LFWDataset(dir=args.lfw_dir,pairs_path=args.lfw_pairs_path,
                     transform=transform),
    batch_size=args.batch_size, shuffle=False, **kwargs)


100%|██████████| 6000/6000 [00:01<00:00, 3384.31it/s]

In [9]:
def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def train(train_loader, model, optimizer, epoch):
    # switch to train mode
    model.train()
    
    pbar = tqdm(enumerate(train_loader))

    top1 = AverageMeter()

    for batch_idx, (data, label) in pbar:
        data_v = Variable(data.cuda())
        target_var = Variable(label)

        # compute output
        prediction = model.forward_classifier(data_v)

        center_loss, model.centers = model.get_center_loss(target_var, args.alpha)

        criterion = nn.CrossEntropyLoss()

        cross_entropy_loss = criterion(prediction.cuda(),target_var.cuda())

        loss = args.center_loss_weight*center_loss + cross_entropy_loss
        #loss = cross_entropy_loss

        # compute gradient and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # update the optimizer learning rate
        adjust_learning_rate(optimizer)

        # log loss value
        #logger.log_value('cross_entropy_loss', cross_entropy_loss.data[0]).step()
        # logger.log_value('total_loss', loss.data[0]).step()

        prec = accuracy(prediction.data, label.cuda(), topk=(1,))
        top1.update(prec[0], data_v.size(0))

        if batch_idx % args.log_interval == 0:
            pbar.set_description(
                'Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t'
                'Train Prec@1 {:.2f} ({:.2f})'.format(
                    epoch, batch_idx * len(data_v), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader),
                    loss.data[0],float(top1.val[0]), float(top1.avg[0])))
            
            file = open('./log_center_loss/Train_Accuracy.txt','a') 
            file.write('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t'
                'Train Prec@1 {:.2f} ({:.2f})\n'.format(
                    epoch, batch_idx * len(data_v), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader),
                    loss.data[0],float(top1.val[0]), float(top1.avg[0])))
            file.close()
    #logger.log_value('Train Prec@1 ',float(top1.avg[0]))

    # do checkpointing
    if not os.path.exists(LOG_DIR):
        os.mkdir(LOG_DIR)
    torch.save({'epoch': epoch + 1,
                'state_dict': model.state_dict(),
                'centers': model.centers},
            '{}/checkpoint_{}.pth'.format(LOG_DIR, epoch))


def test(test_loader, model, epoch):
    # switch to evaluate mode
    model.eval()

    labels, distances = [], []

    pbar = tqdm(enumerate(test_loader))
    for batch_idx, (data_a, data_p, label) in pbar:
        if args.cuda:
            data_a, data_p = data_a.cuda(), data_p.cuda()
        data_a, data_p, label = Variable(data_a, volatile=True), \
                                Variable(data_p, volatile=True), Variable(label)

        # compute output
        out_a, out_p = model(data_a), model(data_p)
        dists = l2_dist.forward(out_a,out_p)#torch.sqrt(torch.sum((out_a - out_p) ** 2, 1))  # euclidean distance
        distances.append(dists.data.cpu().numpy())
        labels.append(label.data.cpu().numpy())

        if batch_idx % args.log_interval == 0:
            pbar.set_description('Test Epoch: {} [{}/{} ({:.0f}%)]'.format(
                epoch, batch_idx * len(data_a), len(test_loader.dataset),
                100. * batch_idx / len(test_loader)))
            file = open('./log_center_loss/Verification_Accuracy.txt','a') 
            file.write('Test Epoch: {} [{}/{} ({:.0f}%)]\n'.format(
                epoch, batch_idx * len(data_a), len(test_loader.dataset),
                100. * batch_idx / len(test_loader)))
            file.close()
    #print(distances)
    labels = np.array([sublabel for label in labels for sublabel in label])
    distances = np.array([subdist for dist in distances for subdist in dist])

    tpr, fpr, accuracy, val, val_std, far = evaluate(distances,labels)
    print('\33[91mTest set: Verification Accuracy: {:.8f}\n\33[0m'.format(np.mean(accuracy)))
    
    
    
    #logger.log_value('Test Accuracy', np.mean(accuracy))

    plot_roc(fpr,tpr,figure_name="roc_test_epoch_{}.png".format(epoch))

def testaccuracy(test_loader,model,epoch):
    # switch to evaluate mode
    model.eval()
    pbar = tqdm(enumerate(test_loader))
    top1 = AverageMeter()
    for batch_idx, (data, label) in pbar:
        data_v = Variable(data.cuda())
        target_value = Variable(label)

        # compute output
        prediction = model.forward_classifier(data_v)
        prec = accuracy(prediction.data, label.cuda(), topk=(1,))
        top1.update(prec[0], data_v.size(0))
        #correct += accuracy(prediction.data, label.cuda(), topk=(1,))[0]*data_v.size(0)
        
        if batch_idx % args.log_interval == 0:
            pbar.set_description(
                'Test Epoch: {} [{}/{} ({:.0f}%)]\t'
                'Test Recognition Prec@1 {:.2f} ({:.2f})'.format(
                    epoch, batch_idx * len(data_v), len(test_loader.dataset),
                    100. * batch_idx / len(test_loader),
                    float(top1.val[0]),float(top1.avg[0]))) 
            file = open('./log_center_loss/Recognition_Accuracy.txt','a') 
            file.write('Test Epoch: {} [{}/{} ({:.0f}%)]\t'
                'Test Recognition Prec@1 {:.2f} ({:.2f})\n'.format(
                    epoch, batch_idx * len(data_v), len(test_loader.dataset),
                    100. * batch_idx / len(test_loader),
                    float(top1.val[0]),float(top1.avg[0])))
            file.close()
    
def plot_roc(fpr,tpr,figure_name="roc.png"):
    import matplotlib.pyplot as plt
    from sklearn.metrics import roc_curve, auc
    roc_auc = auc(fpr, tpr)
    fig = plt.figure()
    lw = 2
    plt.plot(fpr, tpr, color='darkorange',
             lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")
    fig.savefig(os.path.join(LOG_DIR,figure_name), dpi=fig.dpi)


def adjust_learning_rate(optimizer):
    """Updates the learning rate given the learning rate decay.
    The routine has been implemented according to the original Lua SGD optimizer
    """
    for group in optimizer.param_groups:
        if 'step' not in group:
            group['step'] = 0
        group['step'] += 1

        group['lr'] = args.lr / (1 + group['step'] * args.lr_decay)


def create_optimizer(model, new_lr):
    # setup optimizer
    if args.optimizer == 'sgd':
        optimizer = optim.SGD(model.parameters(), lr=new_lr,
                              momentum=0.9, dampening=0.9,
                              weight_decay=args.wd)
    elif args.optimizer == 'adam':
        optimizer = optim.Adam(model.parameters(), lr=new_lr,
                               weight_decay=args.wd, betas=(args.beta1, 0.999))
    elif args.optimizer == 'adagrad':
        optimizer = optim.Adagrad(model.parameters(),
                                  lr=new_lr,
                                  lr_decay=args.lr_decay,
                                  weight_decay=args.wd)
    return optimizer

In [ ]:
#test_display_triplet_distance= True
'''
why test_display_triplet_distance= True in center loss.py?????
'''
test_display_triplet_distance= False
# print the experiment configuration
print('\nparsed options:\n{}\n'.format(vars(args)))
print('\nNumber of Classes:\n{}\n'.format(len(train_dir.classes)))

# # instantiate model and initialize weights



# optionally resume from a checkpoint
if args.resume:
    if os.path.isfile(args.resume):
        print('=> loading checkpoint {}'.format(args.resume))
        checkpoint = torch.load(args.resume)
        args.start_epoch = checkpoint['epoch']
    else:
        checkpoint = None
        print('=> no checkpoint found at {}'.format(args.resume))
print(checkpoint)
#model = FaceModelSoftmax(embedding_size=args.embedding_size,num_classes=len(train_dir.classes),checkpoint=checkpoint)
model = FaceModelCenter(embedding_size=args.embedding_size,num_classes=len(train_dir.classes),checkpoint=None)
if args.cuda:
    print("you are using gpu")
    model.cuda()

optimizer = create_optimizer(model, args.lr)

start = args.start_epoch
end = start + args.epochs

for epoch in range(start, end):
    train(train_loader, model, optimizer, epoch)
    test(test_loader, model, epoch)
    testaccuracy(testaccuracy_loader, model, epoch)
    if test_display_triplet_distance:
        display_triplet_distance_test(model,test_loader,LOG_DIR+"/test_{}".format(epoch))


parsed options:
{'dataroot': '/home/jw4937/datasets/casia/train', 'testdataroot': '/home/jw4937/datasets/casia/test', 'lfw_dir': '/scratch/jw4937/deeplearningdataset/lfw', 'lfw_pairs_path': 'lfw_pairs.txt', 'log_dir': '/scratch/jw4937/logdir_center_loss', 'resume': '/home/jw4937/logdir/run-optim_adam-lr0.001-wd0.0-embeddings512-center0.5-MSCeleb/checkpoint_11.pth', 'start_epoch': 0, 'epochs': 100, 'center_loss_weight': 0.5, 'alpha': 0.5, 'embedding_size': 512, 'batch_size': 64, 'test_batch_size': 64, 'lr': 0.001, 'beta1': 0.5, 'lr_decay': 0.0001, 'wd': 0.0, 'optimizer': 'adam', 'no_cuda': False, 'gpu_id': '0', 'seed': 0, 'log_interval': 10, 'cuda': True}


Number of Classes:
4990

=> no checkpoint found at /home/jw4937/logdir/run-optim_adam-lr0.001-wd0.0-embeddings512-center0.5-MSCeleb/checkpoint_11.pth
None
you are using gpu



0it [00:00, ?it/s]/home/jw4937/Face-recognition/src/model.py:233: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
/home/jw4937/py3.6.3/lib/python3.6/site-packages/ipykernel_launcher.py:79: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number

Train Epoch: 0 [0/48625 (0%)]	Loss: 8.535160	Train Prec@1 0.00 (0.00): : 0it [00:00, ?it/s]/home/jw4937/py3.6.3/lib/python3.6/site-packages/ipykernel_launcher.py:86: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number

Train Epoch: 0 [0/48625 (0%)]	Loss: 8.535160	Train Prec@1 0.00 (0.00): : 1it [00:00,  1.73it/s]
Train Epoch: 0 [0/48625 (0%)]	Loss: 8.535160	Train Prec@1 0.00 (0.00): : 2it [00:00,  2.73it/s]
Train Epoch: 0 [0/48625 (0%)]	Loss: 8.535160	Train Pr

Train Epoch: 0 [3840/48625 (8%)]	Loss: 8.522404	Train Prec@1 0.00 (0.00): : 67it [00:15,  4.32it/s]
Train Epoch: 0 [3840/48625 (8%)]	Loss: 8.522404	Train Prec@1 0.00 (0.00): : 68it [00:15,  4.35it/s]
Train Epoch: 0 [3840/48625 (8%)]	Loss: 8.522404	Train Prec@1 0.00 (0.00): : 69it [00:15,  4.32it/s]
Train Epoch: 0 [3840/48625 (8%)]	Loss: 8.522404	Train Prec@1 0.00 (0.00): : 70it [00:16,  4.34it/s]
Train Epoch: 0 [4480/48625 (9%)]	Loss: 8.526410	Train Prec@1 0.00 (0.00): : 70it [00:16,  4.26it/s]
Train Epoch: 0 [4480/48625 (9%)]	Loss: 8.526410	Train Prec@1 0.00 (0.00): : 71it [00:16,  4.32it/s]
Train Epoch: 0 [4480/48625 (9%)]	Loss: 8.526410	Train Prec@1 0.00 (0.00): : 72it [00:16,  4.34it/s]
Train Epoch: 0 [4480/48625 (9%)]	Loss: 8.526410	Train Prec@1 0.00 (0.00): : 73it [00:16,  4.32it/s]
Train Epoch: 0 [4480/48625 (9%)]	Loss: 8.526410	Train Prec@1 0.00 (0.00): : 74it [00:17,  4.34it/s]
Train Epoch: 0 [4480/48625 (9%)]	Loss: 8.526410	Train Prec@1 0.00 (0.00): : 75it [00:17,  4.33it/s]


Train Epoch: 0 [8320/48625 (17%)]	Loss: 8.527989	Train Prec@1 0.00 (0.01): : 138it [00:31,  4.39it/s]
Train Epoch: 0 [8320/48625 (17%)]	Loss: 8.527989	Train Prec@1 0.00 (0.01): : 139it [00:31,  4.37it/s]
Train Epoch: 0 [8320/48625 (17%)]	Loss: 8.527989	Train Prec@1 0.00 (0.01): : 140it [00:31,  4.39it/s]
Train Epoch: 0 [8960/48625 (18%)]	Loss: 8.528743	Train Prec@1 0.00 (0.01): : 140it [00:32,  4.34it/s]
Train Epoch: 0 [8960/48625 (18%)]	Loss: 8.528743	Train Prec@1 0.00 (0.01): : 141it [00:32,  4.37it/s]
Train Epoch: 0 [8960/48625 (18%)]	Loss: 8.528743	Train Prec@1 0.00 (0.01): : 142it [00:32,  4.38it/s]
Train Epoch: 0 [8960/48625 (18%)]	Loss: 8.528743	Train Prec@1 0.00 (0.01): : 143it [00:32,  4.38it/s]
Train Epoch: 0 [8960/48625 (18%)]	Loss: 8.528743	Train Prec@1 0.00 (0.01): : 144it [00:32,  4.39it/s]
Train Epoch: 0 [8960/48625 (18%)]	Loss: 8.528743	Train Prec@1 0.00 (0.01): : 145it [00:33,  4.38it/s]
Train Epoch: 0 [8960/48625 (18%)]	Loss: 8.528743	Train Prec@1 0.00 (0.01): : 146it

Train Epoch: 0 [12800/48625 (26%)]	Loss: 8.529705	Train Prec@1 0.00 (0.01): : 208it [00:47,  4.40it/s]
Train Epoch: 0 [12800/48625 (26%)]	Loss: 8.529705	Train Prec@1 0.00 (0.01): : 209it [00:47,  4.39it/s]
Train Epoch: 0 [12800/48625 (26%)]	Loss: 8.529705	Train Prec@1 0.00 (0.01): : 210it [00:47,  4.40it/s]
Train Epoch: 0 [13440/48625 (28%)]	Loss: 8.540298	Train Prec@1 0.00 (0.01): : 210it [00:48,  4.37it/s]
Train Epoch: 0 [13440/48625 (28%)]	Loss: 8.540298	Train Prec@1 0.00 (0.01): : 211it [00:48,  4.39it/s]
Train Epoch: 0 [13440/48625 (28%)]	Loss: 8.540298	Train Prec@1 0.00 (0.01): : 212it [00:48,  4.40it/s]
Train Epoch: 0 [13440/48625 (28%)]	Loss: 8.540298	Train Prec@1 0.00 (0.01): : 213it [00:48,  4.39it/s]
Train Epoch: 0 [13440/48625 (28%)]	Loss: 8.540298	Train Prec@1 0.00 (0.01): : 214it [00:48,  4.40it/s]
Train Epoch: 0 [13440/48625 (28%)]	Loss: 8.540298	Train Prec@1 0.00 (0.01): : 215it [00:48,  4.39it/s]
Train Epoch: 0 [13440/48625 (28%)]	Loss: 8.540298	Train Prec@1 0.00 (0.01

...

Train Epoch: 4 [43520/48625 (89%)]	Loss: 7.685095	Train Prec@1 1.56 (0.41): : 685it [03:53,  2.94it/s]
Train Epoch: 4 [43520/48625 (89%)]	Loss: 7.685095	Train Prec@1 1.56 (0.41): : 686it [03:53,  2.93it/s]
Train Epoch: 4 [43520/48625 (89%)]	Loss: 7.685095	Train Prec@1 1.56 (0.41): : 687it [03:53,  2.94it/s]
Train Epoch: 4 [43520/48625 (89%)]	Loss: 7.685095	Train Prec@1 1.56 (0.41): : 688it [03:54,  2.93it/s]
Train Epoch: 4 [43520/48625 (89%)]	Loss: 7.685095	Train Prec@1 1.56 (0.41): : 689it [03:54,  2.94it/s]
Train Epoch: 4 [43520/48625 (89%)]	Loss: 7.685095	Train Prec@1 1.56 (0.41): : 690it [03:55,  2.93it/s]
Train Epoch: 4 [44160/48625 (91%)]	Loss: 7.708110	Train Prec@1 0.00 (0.40): : 690it [03:55,  2.93it/s]
Train Epoch: 4 [44160/48625 (91%)]	Loss: 7.708110	Train Prec@1 0.00 (0.40): : 691it [03:55,  2.94it/s]
Train Epoch: 4 [44160/48625 (91%)]	Loss: 7.708110	Train Prec@1 0.00 (0.40): : 692it [03:55,  2.93it/s]
Train Epoch: 4 [44160/48625 (91%)]	Loss: 7.708110	Train Prec@1 0.00 (0.40

Train Epoch: 4 [48000/48625 (99%)]	Loss: 7.592369	Train Prec@1 0.00 (0.39): : 755it [04:16,  2.94it/s]
Train Epoch: 4 [48000/48625 (99%)]	Loss: 7.592369	Train Prec@1 0.00 (0.39): : 756it [04:17,  2.94it/s]
Train Epoch: 4 [48000/48625 (99%)]	Loss: 7.592369	Train Prec@1 0.00 (0.39): : 757it [04:17,  2.94it/s]
Train Epoch: 4 [48000/48625 (99%)]	Loss: 7.592369	Train Prec@1 0.00 (0.39): : 758it [04:17,  2.94it/s]
Train Epoch: 4 [48000/48625 (99%)]	Loss: 7.592369	Train Prec@1 0.00 (0.39): : 759it [04:18,  2.94it/s]
Train Epoch: 4 [48000/48625 (99%)]	Loss: 7.592369	Train Prec@1 0.00 (0.39): : 760it [04:18,  2.94it/s]

0it [00:00, ?it/s]
Test Epoch: 4 [0/6000 (0%)]: : 0it [00:02, ?it/s]
Test Epoch: 4 [0/6000 (0%)]: : 1it [00:02,  2.27s/it]
Test Epoch: 4 [0/6000 (0%)]: : 3it [00:04,  1.53s/it]
Test Epoch: 4 [0/6000 (0%)]: : 5it [00:06,  1.39s/it]
Test Epoch: 4 [0/6000 (0%)]: : 7it [00:08,  1.23s/it]
Test Epoch: 4 [0/6000 (0%)]: : 9it [00:10,  1.17s/it]
Test Epoch: 4 [0/6000 (0%)]: : 10it [00:10

Test set: Verification Accuracy: 0.56550000




0it [00:00, ?it/s]
Test Epoch: 4 [0/9808 (0%)]	Test Recognition Prec@1 0.00 (0.00): : 0it [00:00, ?it/s]
Test Epoch: 4 [0/9808 (0%)]	Test Recognition Prec@1 0.00 (0.00): : 1it [00:00,  1.01it/s]
Test Epoch: 4 [0/9808 (0%)]	Test Recognition Prec@1 0.00 (0.00): : 3it [00:01,  1.51it/s]
Test Epoch: 4 [0/9808 (0%)]	Test Recognition Prec@1 0.00 (0.00): : 5it [00:02,  1.76it/s]
Test Epoch: 4 [0/9808 (0%)]	Test Recognition Prec@1 0.00 (0.00): : 6it [00:02,  2.03it/s]
Test Epoch: 4 [0/9808 (0%)]	Test Recognition Prec@1 0.00 (0.00): : 7it [00:03,  1.88it/s]
Test Epoch: 4 [0/9808 (0%)]	Test Recognition Prec@1 0.00 (0.00): : 8it [00:03,  2.07it/s]
Test Epoch: 4 [0/9808 (0%)]	Test Recognition Prec@1 0.00 (0.00): : 9it [00:04,  1.90it/s]
Test Epoch: 4 [640/9808 (6%)]	Test Recognition Prec@1 0.00 (0.57): : 9it [00:05,  1.60it/s]
Test Epoch: 4 [640/9808 (6%)]	Test Recognition Prec@1 0.00 (0.57): : 11it [00:05,  1.95it/s]
Test Epoch: 4 [640/9808 (6%)]	Test Recognition Prec@1 0.00 (0.57): : 13it [00:0

Test Epoch: 4 [5760/9808 (58%)]	Test Recognition Prec@1 0.00 (0.57): : 97it [00:45,  2.13it/s]